## **Installing Dependancies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install langchain google-cloud-aiplatform google-auth > /dev/null

# !pip install google-cloud-aiplatform --upgrade
# !pip install google-cloud-bigquery==2.34.2 --user

# # Edit: must install shapely as well
# !pip install -U google-cloud-aiplatform==1.35.0 shapely
!pip install langchain==0.0.306
!pip install google-cloud-aiplatform>=1.33.0
!pip install shapely==1.8.5 --no-cache-dir

!pip install langchain openai tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-bigquery
!pip install langchain openai tiktoken tabulate matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.2
    Uninstalling shapely-2.0.2:
      Successfully uninstalled shapely-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
!pip install langchain==0.0.306
!pip install google-cloud-aiplatform>=1.33.0
!pip install shapely==1.8.5 --no-cache-dir

once Install the libray restart the runtime

In [4]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatVertexAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
import random
import json

In [7]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/kinetic-bot-256820-27a517f7daea.json'

In [8]:
chat = ChatVertexAI(model_name="chat-bison-32k", max_output_tokens=1000,temperature=0.6)

In [9]:
import datetime
import time

In [10]:
import re

In [11]:
import  json

In [18]:
def extract_answer(json_string):
    res = json.loads(json_string)
    f1_score = float(max(res['scores']))
    llm_response = res['labels'][res['scores'].index(max(res['scores']))]
    sys.stdout.flush()

    if f1_score > 0.6:
        print(llm_response)
        print(is_not_valid_email(llm_response))
        if inputs['question_number'] == "1_b" and is_not_valid_email(llm_response):
            return "No match", 1.0

        sys.stdout.flush()
        return llm_response, f1_score
    return "No match", 1.0

In [24]:
def classify_end(text):
    global inputs

    os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/kinetic-bot-256820-27a517f7daea.json'
    chat = ChatVertexAI(model_name="chat-bison-32k", max_output_tokens=1000,temperature=0.6)
    current_date = datetime.date.today()
    # question=knowledge_base[inputs['question_number']]['question']

    instruction=f'''
        Classify the following ANSWER for LABELS ['End Conversation'].


        ANSWER: {text}

        Notes:
        - The JSON object should be valid.
        - If you don't find any matching with these labels, classify the answer as 'No Match'.
        - Only add 'No Match' if no match is found.
        - Please identify whether the ANSWER is an 'End Conversation' prompt. Examples for 'End Conversation' prompts include:
            - "Goodbye"
            - "End chat"
            - "Exit conversation"
            - "I want to this conversation"

        Output: JSON object with the following keys:
        - text: ANSWER.
        - labels: LABELS.
        - scores: The probability that ANSWER is classified as 'End Conversation' or 'No Match'. The total of scores should be one. The probability should be in one decimal point.

        For 'End Conversation' prompts, the probability should be high 1, and for 'No Match', the probability can be set lower based on the model's confidence.


            '''
    # print(instruction)
    prompt = ChatPromptTemplate.from_messages(
            [("system", instruction), ("human", text)])

    # print(prompt)


    messages = prompt.format_messages()
    output = chat(messages)
    response = output.content
    print("classification of input:",response)


    '''
    #extract content from response like ```JSON
                {
                            content
                }
                ```
    '''
    pattern = r'{.*?}'
    match = re.search(pattern, str(response), re.DOTALL)

    if match:
        json_string = match.group()
        # print("output:",json_string)
        res = json.loads(json_string)
        f1_score = float(max(res['scores']))
        llm_response = res['labels'][res['scores'].index(max(res['scores']))]
        # try:
        #     print("one")
        #     return extract_answer(json_string)  #extract the classification output which has highest score
        # except:
        #     try:
        #         print("two")
        #         cleaned_string = str(response).replace('{', '').replace('}', '')
        #         return extract_answer(cleaned_string)
        #     except:
        #         return "No match", 1.0
    else:
        print("three")
        return "No match", 1.0


In [25]:
text="i want to end"

In [26]:
classify_end(text)

classification of input:  ```JSON
{
 "text": "i want to end",
 "labels": [
  "End Conversation"
 ],
 "scores": {
  "End Conversation": 1.0,
  "No Match": 0.0
 }
}
```


JSONDecodeError: ignored

In [ ]:
extract_answer()

In [12]:
inputs = {
              "input_val":"",
              "question_number":"0",
              "user_name":"Narththu",
              "mail_id":"narththu99@gmail.com",
              "event_name":"Birthday",
              "event_type":"Semi private",
              "event_date":"15-10-2023",
              "setup_date":"",
              "add_ons":["Gift bags", "Photography"],
              "service_type":"services",
              "count":"15",
              "Services":["Fragrance Discovery", "Skin Consultation","One-on-One make up touch ups"],
              "Workshop":"Workshop is not available for his event.",
              'chat_history': [],
              "chat_backup":"",
              'All_chat' : {}
              }

In [13]:
default_history = "This is the details about the event user want to conduct. General information about the client's event are event name, user name, email ID, event date, event type (private or semi-private), guest count, visitation date, service type(Services or Services and workshop. Service type is only for semi-private events), available services and workshops, as well as available add-ons (One or more add_ons from Catering, Gift bags, Photography, Head shots, Additional beauty services, and other customization options)."\
                    "\nHis Event Name: {event_name}\nHis Name: {user_name}\nHis Email ID: {mail_id}\nHis Event Date: {event_date}\nHis Event Type: {event_type}." \
                      " If his event_type is private entire space is allocated for his event. So general public is not allowed to use the space. But if his event_type semi private event a special dedicated section is allocated for the event within the venue while the rest of the space remains accessible to the general public. " \
                      "Guest Count of his event is: {count}. \n" \
                      "He will Visit for your place on: {setup_date}\nHis Service Type is: {service_type}\n"\
                      "Services he can access:{Services}\nAdd ons he selected :{add_ons}"\
                      "\nServices available for him are {Services}\n{Workshop}. If he is not capable to get workshop he can change the service type to Services and Workshops. Or otherwise he can change the event type to private" \
                      " \nAdd-ons he selected are: {add_ons} and Now he can only access these now.But he can able to choose the add_ons such as Catering,Gift bags, Photography,Head shots, Additional beauty services and other customization add_ons if he want.\n".format(
        **inputs)

In [14]:
print("Defaullt history is, \n",default_history)

Defaullt history is, 
 This is the details about the event user want to conduct. General information about the client's event are event name, user name, email ID, event date, event type (private or semi-private), guest count, visitation date, service type(Services or Services and workshop. Service type is only for semi-private events), available services and workshops, as well as available add-ons (One or more add_ons from Catering, Gift bags, Photography, Head shots, Additional beauty services, and other customization options).
His Event Name: Birthday
His Name: Narththu
His Email ID: narththu99@gmail.com
His Event Date: 15-10-2023
His Event Type: Semi private. If his event_type is private entire space is allocated for his event. So general public is not allowed to use the space. But if his event_type semi private event a special dedicated section is allocated for the event within the venue while the rest of the space remains accessible to the general public. Guest Count of his event 

In [ ]:
instruction =  f"""
You are a dedicated customer service chatbot, designed to be helpful, respectful, and truthful.

1. **USER QUERY** represents the input from the user.

2. **RESPONSE SOURCES**:
   - **CHAT_HISTORY**: Contains past conversations or specific details about the user's events.
   - **CHAT_HISTORT**: Contains general information about the client's event, including event name, user name, email ID, event date, event type (private or semi-private), guest count, visitation date, service type(Services or Services and workshop. Service type is only for semi-private events), available services and workshop is available or not, as well as available add-ons (One or more add_ons from Catering, Gift bags, Photography, Head shots, Additional beauty services, and other customization options).
   - **COMMON KNOWLEDGE**: General information not specific to the user's event or the channel.

3. **RESPONSE GUIDELINES**:
   - If the USER
   QUERY pertains to the user's specific event, prioritize responding from the CHAT_HISTORY.
   - If the USER QUERY is about general channel activities not specific to the user's event, refer to the CONTEXT to generate a response.
   - For questions outside the user's event or the CONTEXT, utilize COMMON KNOWLEDGE.
   - Always ensure the user's safety and provide meaningful answers. Avoid empty or nonsensical replies.
   - If the USER QUERY is ambiguous or unclear, request clarification from the user.
   - When users express gratitude, greetings, or positive sentiments, respond warmly and courteously.

Your primary objective is to assist users effectively and ensure their safety.

"""
system_pompt = instruction.strip()
print(system_pompt)

You are a dedicated customer service chatbot, designed to be helpful, respectful, and truthful.

1. **USER QUERY** represents the input from the user.

2. **RESPONSE SOURCES**:
   - **CHAT_HISTORY**: Contains past conversations or specific details about the user's events.
   - **CHAT_HISTORT**: Contains general information about the client's event, including event name, user name, email ID, event date, event type (private or semi-private), guest count, visitation date, service type(Services or Services and workshop. Service type is only for semi-private events), available services and workshop is available or not, as well as available add-ons (One or more add_ons from Catering, Gift bags, Photography, Head shots, Additional beauty services, and other customization options).
   - **COMMON KNOWLEDGE**: General information not specific to the user's event or the channel.

3. **RESPONSE GUIDELINES**:
   - If the USER QUERY pertains to the user's specific event, prioritize responding from t

In [ ]:
context = f"""
We are an event organising channel named "Atelier Beauty channel" that offers 2 types of events such as semi-private events and private events.
For Semi-private events 2-18 is the guest count limit. For private event type 2-125 is the guest count limit. If User's guest is greater than 18 reccomand him to choose private event type.

There are 3 Services provide by us such as: 1.Fragrance Discovery service: here we will embark on a journey with our Fragrance Expert to discover our signature CHANEL scent. For more details: https://atelierbeaute.chanel.com/discover/groups#fragrance-discovery. 2.Skin Consultation service : Here expert guidance tailored to your guests’ personal skin concerns. For more details: https://atelierbeaute.chanel.com/discover#skin-consultation. 3.Makeup touch-ups service: Conducted by our Atelier Makeup Artists. For more details: https://atelierbeaute.chanel.com/discover/groups#makeup-happy-hour.\n\n

Semi-private events are conducted during our regular operating hours, allowing guests to have a special dedicated section for the event within the venue while the rest of the space remains accessible to the general public(So the entire space is not used for the event). The duration of these events can be from 1 to 2 hours. The number of guests for semi-private events can range from 2 up to 18.\n\n

For the SEMI-PRIVATE type events  and the number of guests/attendees/head_count are less than 6 then, visit our page- https://atelierbeaute.chanel.com/discover/groups where you can book an experience directly on our website. Be sure to read the “COMMON QUESTIONS” at the bottom of the page for more information on booking up to 6 guests.\n\n
For semi-private type events there are 2 service-types available. They are Mix-and-Match-services and Services-&-Workshops. You have to choose whether you want “Mix-and-Match Services” or “Services and Workshops”.\n\
If you choose “Mix-and-Match Services” then you can only get 2-3 Services from Fragrance Discovery, Skin Consultation and One-on-One Makeup. That means you can’t get workshops.\n
If you choose “Services and Workshops” you can get both mix and match services and also hands-on experience with an Atelier Makeup Artist who will teach you a range of techniques such as Mastering the CHANEL Makeup Look or Day-to-Night Beauty. But based on the number of guests the offering mix and match services and workshops differ.\n\n

Private events at Atelier, are using the whole place to yourself so no other guests from the public can’t be accessible by the general public. These events can last 2 to 3 hours. You can invite guests from 2 to 125 people. For the private events we offer the combination of “Mix-And-Match services” and “Workshops” as default. So no need to choose a type of service. But based on the Number of guests the services you can access and workshops are differ.\n\n
For the service type of "Services" also known as “Mix and Match Services” for “Semi-private” event type with 7-12 guests you can only get Fragrance Discovery and One-on-One Makeup touch ups services. The cost start at $2520 for a 90-minute event which includes both services for all your guests and a glass of champagne.\n\n
For the service type of "Services" also known as “Mix and Match Services” for “Semi-private” event type with 13-18 guests you can only get Fragrance Discovery, Skin Consultation and One-on-One Makeup touch ups services And also the cost start at $4230 for a 120-minute event which includes all three services for all your guests and a glass of champagne.\n\n
For the service type “Services and workshops” for “Semi-private” event type with 7-12 guests you can only get the Fragrance Discovery service. And additionally hands-on Workshop in a theme of your choice. The prices start at $2520 for a 90-minute event which includes both the Fragrance Discovery and the Workshop for all your guests, and a glass of champagne.\n\n
For the service type “Services and workshops” for “Semi-private” event type with 13-18 guests you can only get the Fragrance Discovery and Skin Consultation services. And additionally hands-on Workshop in a theme of your choice. The prices start at $4470 for a 2 hour event which includes both services and the Workshop for all your guests, and a glass of champagne.\n\n
For the Private events with 2-24 guests the available “Mix and Match” services are Fragrance Discovery, Skin Consultation and One-on-One Makeup touch ups services. Additionally, workshops offer your guests a hands-on experience with an Atelier Makeup Artist who will teach you a range of techniques such as Mastering the CHANEL Makeup Look or Day-to-Night Beauty. The prices start at $11140 for a 2 hour event which includes all three services for all your guests, plus champagne and/or wine.\n\n
For the Private events with 25-30 guests the available “Mix and Match” services are Fragrance Discovery, Skin Consultation and One-on-One Makeup touch ups services. Additionally, workshops offer your guests a hands-on experience with an Atelier Makeup Artist who will teach you a range of techniques such as Mastering the CHANEL Makeup Look or Day-to-Night Beauty. The prices start at $13340 for a 3-hour event which includes all three services for all your guests, plus champagne and/or wine.\n\n
For the Private events with 31-50 guests the available “Mix and Match” services are Fragrance Discovery and then guests will have a choice of second service of selecting one from the Skin Consultation/One-on-One Makeup touch ups.  Additionally, workshops offer your guests a hands-on experience with an Atelier Makeup Artist who will teach you a range of techniques such as Mastering the CHANEL Makeup Look or Day-to-Night Beauty. The prices start at $18,500 for a 3-hour event which include two services for all your guests, plus champagne and/or wine.\n\n
For the Private events with 25-30 guests the available “Mix and Match” services are Fragrance Discovery, Skin Consultation and One-on-One Makeup touch ups on a first come first served basis. Additionally, workshops offer your guests a hands-on experience with an Atelier Makeup Artist who will teach you a range of techniques such as Mastering the CHANEL Makeup Look or Day-to-Night Beauty. The prices start at $27600 which include access to all three services for all your guests, plus champagne and/or wine.\n\n
Any add-ons such as catering, photography, Additional beauty services, gift bags and other can be added for all types of events with an additional charge.

"""

In [ ]:
reply_for_none = [
            "Sorry, I couldn't grasp that. Could you please clarify?",
            "I apologize, but I'm not sure what you mean. Can you rephrase?",
            "I didn't quite catch that. Can you please explain further?",
            "I'm sorry, that was unclear to me. Can you elaborate?",
            "Pardon me, I didn't understand. Could you say that differently?",
            "My apologies, I couldn't follow. Can you clarify your statement?",
            "Sorry, that went over my head. Can you explain in another way?",
            "Apologies for the confusion. Can you provide more detail?",
            "I couldn't make sense of that. Could you please rephrase your question?"]

In [ ]:
while True:
  question = input("User: ")
  human = f"""
  CHAT_HISTORY : {default_history}
  CONTEXT : {context}
  USER QUERY : {question}
  """
  human = human.strip()
  prompt = ChatPromptTemplate.from_messages([("system", instruction), ("human",  human)])
  messages = prompt.format_messages()

  # output = chat(messages)
  # response = output.content


  attempts = 0
  while attempts < 5:
      print("Attempt for the ", question, " is ", attempts + 1)
      output = chat(messages)
      response = output.content
      if not response.isspace() and response:  # Check if response is not just whitespace and is not empty
          break
      attempts += 1

  if response.isspace() or not response:  # If response contains only whitespace or is empty
      # Get a random response from the list
      response = random.choice(reply_for_none)
  print("BOT: ",response,"\n")

User: Do you have date in your mind
Attempt for the  Do you have date in your mind  is  1
BOT:   15-10-2023 is the date of your event.  



In [17]:
text=""
x=f"""Identify Number or Question  from the following ANSWER.

QUESTION: May I know approximately how many guests are we expecting for your event?
ANSWER: {text}
Notes:
  • The JSON object should be valid.
  • classify the value to the "Number" and questions to "Question". The value should be whole number.
  • Please classify whether the ANSWER is a 'Question'. Examples for 'Question' are:
            What time is it?
            Is this a question?
            Details please?
            Recommend?
            Can I contact you
  • If you don't find any value or if you don't find Question from the ANSWER, Classifiy ANSWER as No match and append this to LABELS and put high probability for it.
  • Only add No match if value is not found or Question is not found.
  • Don't classify as Number  Instead assign the value of whole number.



Output: JSON object with the following keys:
  • text: ANSWER.
  • labels: [Should be a Number or Question or No match].
  • scores: The probability of the identification of number or Question from the ANSWER. This should be a list with the length same as labels. The probability score should be a float in one decimel point.
"""

import sys
import re
classify_answer(x,text)

Identify Number or Question  from the following ANSWER.

QUESTION: May I know approximately how many guests are we expecting for your event?
ANSWER: can i know you
Notes:
  • The JSON object should be valid.
  • classify the value to the "Number" and questions to "Question". The value should be whole number.
  • Please classify whether the ANSWER is a 'Question'. Examples for 'Question' are: 
            What time is it?
            Is this a question?
            Details please?
            Recommend?
            Can I contact you
  • If you don't find any value or if you don't find Question from the ANSWER, Classifiy ANSWER as No match and append this to LABELS and put high probability for it.
  • Only add No match if value is not found or Question is not found.
  • Don't classify as Number  Instead assign the value of whole number.



Output: JSON object with the following keys:
  • text: ANSWER.
  • labels: [Should be a Number or Question or No match].
  • scores: The probability o

In [ ]:
prompt = ChatPromptTemplate.from_messages([("system", x), ("human",  text)])
messages = prompt.format_messages()

In [ ]:
print(messages)

[SystemMessage(content='Classify the following ANSWER for LABELS [\'Book Another Event\',\'End the Conversation\'].\nQUESTION: Would you like to Book Another Event or to End this conversation?\nANSWER: hi\nNotes:\n  • The JSON object should be valid.\n  • If you don\'t found any matching with these labels. Classify answer as "Current Conversation" and append this to LABELS and put high possibility for it.\n  • Only add No mathc if no match found.\nOutput: JSON object with the following keys:\n  • text: ANSWER.\n  • labels: LABELS.\n  • scores: The probability to ANSWER is being classified as the label for each label in LABELS. This should be a list with the length same as LABELS. Total of scores should be one. The probability should be a float in one decimal point.'), HumanMessage(content='hi')]


In [ ]:
output = chat(messages)
output

AIMessage(content=' ```JSON\n{\n  "text": "hi",\n  "labels": [\n    "Current Conversation",\n    "Book Another Event",\n    "End the Conversation"\n  ],\n  "scores": [\n    0.6,\n    0.2,\n    0.2\n  ]\n}\n```')

Classify the following ANSWER for LABELS ['Book Another Event','End the Conversation'].
QUESTION: Would you like to Book Another Event or to End this conversation?
ANSWER: hi
Notes:
  • The JSON object should be valid.
  • If you don't found any matching with these labels. Classify answer as "Current Conversation" and append this to LABELS and put high possibility for it.
  • Only add No mathc if no match found.
Output: JSON object with the following keys:
  • text: ANSWER.
  • labels: LABELS.
  • scores: The probability to ANSWER is being classified as the label for each label in LABELS. This should be a list with the length same as LABELS. Total of scores should be one. The probability should be a float in one decimal point.
input_variables=[] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Classify the following ANSWER for LABELS [\'Book Another Event\',\'End the Conversation\'].\nQUESTION: Would you like to Book Another Event or to End this

In [15]:
def classify_answer(instruction, text):
    # os.environ[
    #     'GOOGLE_APPLICATION_CREDENTIALS'] = '/interplay_v2/public/private/DATA/PROJECTS/FAQ_Bot/KEYS/Bison/kinetic-bot-256820-27a517f7daea.json'
    # chat = ChatVertexAI(model_name="chat-bison-32k", max_output_tokens=1000)

    # current_date = datetime.date.today()
    print(instruction)
    prompt = ChatPromptTemplate.from_messages(
        [("system", instruction.replace("{text}", text)), ("human", text)])
    print(prompt)
    messages = prompt.format_messages()
    sys.stdout.flush()
    output = chat(messages)
    response = output.content
    pattern = r'{.*?}'
    match = re.search(pattern, str(response), re.DOTALL)
    print(match.group())
    if match:
        json_string = match.group()
        print(json_string)
    #     try:
    #         return extract_answer(json_string)
    #     except:
    #         try:
    #             cleaned_string = str(response).replace('{', '').replace('}', '')
    #             return extract_answer(cleaned_string)
    #         except:
    #             return "No match", 1.0
    # else:
    #     return "No match", 1.0